<h3> Imports

In [105]:
#Pytorch
import torch
import torch.nn.functional as F 

#Transformers
from transformers import pipeline

#Maths
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import evaluate

#Sckit 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

#NLP
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#Others
import seaborn as sns
import time
import string
import os
import random

In [18]:
#Torchtext
import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

Beaucoup de problèmes avec l'installation de torchtext. Si jamais ça bug faire les commmandes suivantes: 
<li> pip uninstall torchvision
<li> pip uninstall torchtext
<li> pip install torchtext

<h3>Loading Data

This is the data preprocessed in the TFIDF notebook.

In [19]:
data = pd.read_csv('datasets/news-article-categories-clean.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6871 entries, 0 to 6870
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6871 non-null   object
 1   title     6871 non-null   object
 2   body      6871 non-null   object
dtypes: object(3)
memory usage: 161.2+ KB


In [75]:
id2label = {0: "ARTS & CULTURE", 1: "BUSINESS",2: "COMEDY", 3: "CRIME",4: "EDUCATION", 5: "ENTERTAINMENT",6: "ENVIRONMENT", 7: "MEDIA",8: "POLITICS", 9: "RELIGION", 10: "SCIENCE", 11: "SPORTS",12: "TECH", 13: "WOMEN"}
label2id = {"ARTS & CULTURE" : 0,"BUSINESS" : 1 ,"COMEDY" : 2,"CRIME" : 3, "EDUCATION" : 4,"ENTERTAINMENT" : 5, "ENVIRONMENT" : 6, "MEDIA" : 7 ,"POLITICS" : 8, "RELIGION" : 9, "SCIENCE" : 10, "SPORTS" : 11 ,"TECH" : 12 , "WOMEN" : 13}

<h2> BART

In [135]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to C:/Users/julie/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [136]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [140]:
dataset["train"]

Dataset({
    features: ['label', 'text'],
    num_rows: 650000
})

<h3> Pipeline

In [29]:
### TASK ###
task = "zero-shot-classification"
# task = "text-classification"

### MODEL ###
model = "facebook/bart-large-mnli"

pipe = pipeline(task, model)

<h4> Pipeline Test

In [101]:
sequence = ['Trump is saying nonsense again', "Shops are going under"]
labels = ["energy", "retail", "politics", "gaming"]
res = pipe(sequence, labels)

print(sequence[0])
res[0]

Trump is saying nonsense again


{'sequence': 'Trump is saying nonsense again',
 'labels': ['gaming', 'politics', 'retail', 'energy'],
 'scores': [0.5610557198524475,
  0.39181092381477356,
  0.030370013788342476,
  0.01676333323121071]}

In [102]:
labels = data['category'].unique()
labels

array(['ARTS & CULTURE', 'BUSINESS', 'COMEDY', 'CRIME', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'MEDIA', 'POLITICS', 'RELIGION',
       'SCIENCE', 'SPORTS', 'TECH', 'WOMEN'], dtype=object)

<h4> Target Vector

In [103]:
title = data.iloc[:, 1].values
X = data.iloc[:, 2].values
y = data.iloc[:, 0].values

<h4> TF-IDF Matrix

In [117]:
tf_idf = False

In [118]:
if tf_idf:
    # Building a TF IDF matrix out of the corpus of reviews
    td = TfidfVectorizer(max_features = 4500)
    X_vect = td.fit_transform(X).toarray()
    title_vect = td.fit_transform(title).toarray()
else:
    X_vect = X
    title_vect = title

<h4> Train / Test Split

In [119]:
# Splitting into training & test subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size = 0.3,random_state = 0)

<h4> Results

In [120]:
title_test = title[0:10]
body_test = X[0:2]

In [121]:
results = pipe(title_test.tolist(),labels)

In [39]:
results2 = pipe(body_test.tolist(),labels)

In [122]:
y_pred = []
for dic in results:
    y_pred.append(dic['labels'][0])

In [41]:
y_pred2 = []
for dic in results2:
    y_pred2.append(dic['labels'][0])

In [123]:
print('____________ PREDICTED CATEGORIES FROM TITLE ____________')
for cat in y_pred:
    print(cat)
print('____________ PREDICTED CATEGORIES FROM BODY ____________')
for cat in y_pred2:
    print(cat)
print('____________ REAL CATEGORIES ____________')
for cat in y[0:10]:
    print(cat)

____________ PREDICTED CATEGORIES FROM TITLE ____________
CRIME
ENTERTAINMENT
MEDIA
ENTERTAINMENT
COMEDY
MEDIA
ENTERTAINMENT
ARTS & CULTURE
BUSINESS
ENTERTAINMENT
____________ PREDICTED CATEGORIES FROM BODY ____________
TECH
ENTERTAINMENT
____________ REAL CATEGORIES ____________
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE
ARTS & CULTURE


<h3> Fine-tuning BART

In [124]:
from transformers import TrainingArguments


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

metric = evaluate.load("accuracy")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [125]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [131]:
from transformers import Trainer

xtrain, xval, ytrain, yval = train_test_split(X_train, y_train, test_size = 0.3,random_state = 0)
train = pd.concat([pd.DataFrame(xtrain), pd.DataFrame(ytrain)], axis = 1)
val = pd.concat([pd.DataFrame(xval), pd.DataFrame(yval)], ignore_index=True)

In [134]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=pd.DataFrame(xtrain),
    eval_dataset=pd.DataFrame(xval),
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\julie\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3366
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1263
  Number of trainable parameters = 66362880


  0%|          | 0/1263 [00:00<?, ?it/s]

KeyError: 2180

<h2> BERT

<h3> Tokenization

In [106]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [129]:
test = ["This is a test", "Does it work with lists?"]

In [130]:
encoding = tokenizer.encode_plus(test, add_special_tokens = True,    
                                    truncation = True, 
                                    padding = "max_length", 
                                    return_attention_mask = True, 
                                    return_tensors = "pt")

TO COMPLETE

<h2> DistilBERT

<h3> Test

In [89]:
from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

inputs = tokenizer("I really didn't like this movie, I was bored to death towards the end", return_tensors="pt")
# inputs = tokenizer("I really loved this movie, the art style was fascinating, I really enjoyed myself", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
predicted_class_id

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

0

<h3> Configuration

In [91]:
from transformers import DistilBertModel, DistilBertConfig

# Initializing a DistilBERT configuration
configuration = DistilBertConfig()

# Initializing a model from the configuration
model = DistilBertModel(configuration)

# Accessing the model configuration
configuration = model.config 

<h3> Tokenization

In [92]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
inputs 

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

<h3> DistilBert for sequence classification

In [95]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs)
# loss = outputs.loss
# logits = outputs.logits

<h2> BertTopic